In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from datetime import datetime

In [8]:
def get_blog_text_from_aidb(soup):
    blog_text = soup.select_one('div[class="post_content clearfix"]').text
    return blog_text

In [9]:
def clean_blog_text(blog_text: str) -> str:
    # 改行やスペースを取り除く
    cleaned_text = re.sub(r'\s+', ' ', blog_text).strip()

    # "マイページに保存"という言葉が2回blog_text出てくるか確認する
    if cleaned_text.count("マイページに保存") == 2:
        # 2回出てくる場合は、”マイページに保存”という言葉の前とあとを取り除く。
        start_index = cleaned_text.find("マイページに保存") + len("マイページに保存")
        end_index = cleaned_text.rfind("マイページに保存")
        
        # ”マイページに保存”という言葉の前とあとを取り除く
        cleaned_text = cleaned_text[start_index:end_index]

    return cleaned_text    

In [10]:
index_df = pd.read_excel("../data/aidb/index.xlsx")
index_df

,release_date,title,url
0,2023/10/27,GPT-4などLLMのコード生成能力にデバッグ機能を追加する『SELF-DEBUGGING（...,https://aiboom.net/archives/57709
1,2023/10/26,LLMに非線形的な思考を与えてCoTを上回る性能を引き出す手法『IEP』と実行プロンプト C...,https://aiboom.net/archives/57628
2,2023/10/25,"Natureなどの論文約4,800本でGPT-4による査読能力が検証され、「LLMは査読にも...",https://aiboom.net/archives/57449
3,2023/10/24,DALL-E 3はどうしてユーザーの意図を正確に汲み取ることができるのか？OpenAIが論文で発表,https://aiboom.net/archives/57310
4,2023/10/23,LLMにベートーヴェンなど特定の人物の行動や感情を模倣させる、イタコのような技術『Chara...,https://aiboom.net/archives/57223
...,...,...,...
147,2021/11/19,【Facebook AI】2人競技における選手の動きをAIでシミュレーション（AI×スポーツ...,https://aiboom.net/archives/49274
148,2021/11/1,ボクシング・パンチを機械学習 AIによるトレーニングコーチの仕事【AI×スポーツ】（論文解説）,https://aiboom.net/archives/49248
149,2021/10/12,AIで稲の病気を検出 害虫の被害を防止する自動化テクノロジー【AI×農業】（論文解説）,https://aiboom.net/archives/48905
150,2021/10/5,1枚のスケッチから世に無い新しい写真を作り出すことが可能に（AI論文解説）,https://aiboom.net/archives/48923


In [11]:
for i, row in index_df.iterrows():
    sleep(1)
    response = requests.get(row["url"])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # テキスト抽出
    blog_text = get_blog_text_from_aidb(soup)

    # クレンジング
    cleaned_blog_text = clean_blog_text(blog_text)

    # 'release_date'を解析して、8桁の年月日形式に変換
    formatted_date = datetime.strptime(row['release_date'], '%Y/%m/%d').strftime('%Y%m%d')

    # ファイル名
    filename = f"{formatted_date}_{row['url'].split('/')[-1]}.txt"

    # ファイルにテキストを保存する
    with open(f"../data/aidb/pages/{filename}", 'w', encoding='utf-8') as file:
        file.write(cleaned_blog_text)